In [60]:
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np
import autokeras as ak
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [61]:
clf = ak.StructuredDataClassifier(
    loss=tf.keras.losses.binary_crossentropy,
    overwrite=True,
)
data = pd.read_csv('train.csv')
df_train = data.drop(['Name',],axis=1)
df_t= pd.read_csv('test.csv')
df_test = df_t.drop(['Name',],axis=1)

In [62]:
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [63]:
mapping_cabin = {
    "A" : 0,
    "B":1,
    "C":3,
    "D":4,
    "E":5,
    "F":6,
    "G":7,
    "T":8
 }

In [64]:
def preprocess(df):
    df["Total"] =  df["ShoppingMall"] + df["VRDeck"] + df["Spa"] + df["RoomService"] + df["FoodCourt"]
    mean_total = df["Total"].mean()
    median_Age = df["Age"].median()
    for i in range(len(df)):
        if df["VIP"][i] == np.NaN:
            if df["Total"] > mean_total:
                df["VIP"][i] = 1
            else :
                df["VIP"][i] = 0
        if df["CryoSleep"] is np.NaN:
            if df["Total"][i] > mean_total:
                df["CryoSleep"][i] = 0
            else :
                df["CryoSleep"][i] = 1
        if ((df["Cabin"][i] is not np.NaN) and (df["Cabin"][i] is not int)) :
            df["Cabin"][i] = mapping_cabin[df["Cabin"][i][0]]
    df["Age"].fillna(median_Age)
            
        

In [65]:
preprocess(df_train)

In [66]:
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Total
0,0001_01,Europa,False,1,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0.0
1,0002_01,Earth,False,6,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,736.0
2,0003_01,Europa,False,0,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,10383.0
3,0003_02,Europa,False,0,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,5176.0
4,0004_01,Earth,False,6,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,1091.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,0,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,8536.0
8689,9278_01,Earth,True,7,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,0.0
8690,9279_01,Earth,False,7,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,1873.0
8691,9280_01,Europa,False,5,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,4637.0


In [67]:
df_train["Destination"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 8693 entries, 0 to 8692
Series name: Destination
Non-Null Count  Dtype 
--------------  ----- 
8511 non-null   object
dtypes: object(1)
memory usage: 68.0+ KB
